# Download Searchanalytics Data
Herunterladen von Google Search Console Suchanalyse-Daten mit dem Python Package `searchconsole` [github link](https://github.com/joshcarty/google-searchconsole)

In [ ]:
import searchconsole

Verifizieren des API Zugriffs.

→ Wenn ihr das Notebook auf Binder.org ausführt, werden eure Credentials natürlich nicht gespeichert!

## Authentifizierung - Notebook auf mybinder.org
Wenn ihr das Notebook auf mybinder.org ausführt, öffnet die `.authenticate` Methode kein neues Fenster für die Authentifizierung. Ich gehe daher den Umweg über `pydata_google_auth`. 

In [ ]:
import json
import pydata_google_auth
from pydata_google_auth import cache
from googleapiclient.discovery import build
from searchconsole.account import Account

with open('client_secrets.json') as f:
    cs = json.load(f)

credentials = pydata_google_auth.get_user_credentials(scopes=['https://www.googleapis.com/auth/webmasters.readonly'],
                                                      client_id=cs['installed']['client_id'],
                                                      client_secret=cs['installed']['client_secret'],
                                                      credentials_cache=cache.REAUTH,
                                                      auth_local_webserver=False)

In [ ]:
service = build('webmasters', 'v3', credentials=credentials)

account = Account(service, credentials)

## Authentifizierung - Lokales Notebook
Bei einem lokalen Notebook kann einfach fortgefahren werden.

In [ ]:
account = searchconsole.authenticate(client_config='client_secrets.json')

## Daten abfragen

Liste der verifizierten Properties in dem Account

In [ ]:
account.webproperties

Webproperty Object mit dem wir den Query erstellen.

In [ ]:
webproperty = account['https://www.serienjunkies.de/']

Query erstellen und ausführen.

In [ ]:
query = webproperty.query.range(start='2019-03-18', stop='2019-03-24') \
    .dimension('query')

In [ ]:
data = query.get()

So sieht das ganze als Pandas DataFrame aus. Hiermit kann man nun alles mögliche anstellen.

In [ ]:
df = data.to_dataframe()
df.head()

Eine Abfrage mit Filter.

In [ ]:
df = webproperty.query.range(start='2019-03-18', stop='2019-03-24') \
    .dimension('page','query') \
    .filter('page', 'https://www.serienjunkies.de/', 'equals') \
    .get() \
    .to_dataframe()
df.head()

Export der Daten als CSV. Die Datei sollte heruntergeladen werden können. ✌️😉

In [ ]:
df.to_csv('gsc_searchanalytics_output.csv', header=True, index=False)